# Recreating transit data from MP 01

In [ ]:
# import libraries
import numpy as np
import pandas as pd

In [3]:
# import fares
fares = pd.read_excel('data/2022_fare_revenue.xlsx')
# We only want the "funds earned during period"
fares = fares[fares['Expense Type']=="Funds Earned During Period"]
# renaming columns for simplicity
fares.rename({'NTD ID':'id','Mode':'mode','Total Fares':'fares','Agency Name':'agency'},axis=1,inplace=True)
# removing extra cols for initial analysis
fares.drop(['Expense Type','State/Parent NTD ID',
            'Reporter Type','Reporting Module','TOS',
            'Passenger Paid Fares','Organization Paid Fares'],axis=1,inplace=True)

In [89]:
# import expenses
# want to specify that NTD ID is an integer
dtypes = {'NTD ID':'int'}
expenses = pd.read_csv('data/2022_expenses.csv',dtype=dtypes)
# renam cols
expenses.rename({'NTD ID':'id','Mode':'mode','Total':'expenses'},axis=1,inplace=True)
# We want the expenses summed by ntd id and mode
expenses = expenses[['id','mode','expenses']].groupby(['id','mode','expenses']).sum('expenses').reset_index()
# need to fix this column
expenses['expenses'] = expenses['expenses'].str.replace(',','').astype(int)
# want all of expenses in one
expenses = expenses.groupby(['id','mode']).sum('expenses').reset_index()

In [105]:
# create financials table by merging expenses onto fares table
financials = fares.merge(expenses,how='inner',on=['id','mode']).dropna()

In [106]:
financials

,id,agency,mode,fares,expenses
0,1,King County Department of Metro Transit,CB,5216912,0
1,1,King County Department of Metro Transit,DR,740726,60540676
2,1,King County Department of Metro Transit,DR,91601,60540676
3,1,King County Department of Metro Transit,FB,1715265,8899659
4,1,King County Department of Metro Transit,LR,29386480,0
...,...,...,...,...,...
1343,99423,City of Glendale,MB,238992,10012370
1344,99424,City of Pasadena,DR,18305,1752583
1345,99424,City of Pasadena,MB,482055,6777893
1346,99425,Pomona Valley Transportation Authority,DR,80294,3202374


In [108]:
# we can kinda do this since we already know expenses are unique from how we imported it
# but I'm just leaving it as its own thing
financials.groupby(['id','agency','mode','expenses']).sum('fares').reset_index()

,id,agency,mode,expenses,fares
0,1,King County Department of Metro Transit,CB,0,5216912
1,1,King County Department of Metro Transit,DR,60540676,832327
2,1,King County Department of Metro Transit,FB,8899659,1715265
3,1,King County Department of Metro Transit,LR,0,29386480
4,1,King County Department of Metro Transit,MB,671653000,56846337
...,...,...,...,...,...
1168,99423,City of Glendale,DR,1011334,13628
1169,99423,City of Glendale,MB,10012370,238992
1170,99424,City of Pasadena,DR,1752583,18305
1171,99424,City of Pasadena,MB,6777893,482055


In [6]:
# Ensure we pick the right sheet when reading in trips
trips = pd.read_excel('data/ridership.xlsx',sheet_name='UPT')
# only want certain columns + rename cols
trips = trips[trips['Mode/Type of Service Status']== 'Active'].drop(['Legacy NTD ID','Reporter Type',
                                                                     'Mode/Type of Service Status','UACE CD','TOS'],
                                                                     axis=1)
trips.rename({'NTD ID':'id','Agency':'agency','3 Mode':'mode_3',
             'UZA Name':'metro_area','Mode':'mode'},axis=1,inplace=True)

In [7]:
# Pivot the table since the months are used as columns right now
trips_melted = pd.melt(trips,
                       id_vars=['id','agency','metro_area','mode','mode_3'], # These columns will stay the same
                       var_name='month', # Newly produced columns will be renamed 'month'
                       value_name='UPT' # The values will be in the UPT column
                       ).dropna()
# ensure month is registered as datetime for ease of future anaylsis
trips_melted['month'] = pd.to_datetime(trips_melted['month'])
# Need to also make sure id is still an int and not a float now with NULL values removed
trips_melted['id'] = trips_melted['id'].astype('int')

/tmp/ipykernel_19843/3082388799.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  trips_melted['month'] = pd.to_datetime(trips_melted['month'])


In [8]:
# Testing it visually with results from the R table
trips_melted[(trips_melted['agency']=='Regional Transportation Authority of Pima County')&(trips_melted['mode']=='DR')].sort_values(by='month',ascending=False).head(3)

,id,agency,metro_area,mode,mode_3,month,UPT
425461,91122,Regional Transportation Authority of Pima County,"Tucson, AZ",DR,Bus,2024-07-01,10811.0
423891,91122,Regional Transportation Authority of Pima County,"Tucson, AZ",DR,Bus,2024-06-01,9617.0
422321,91122,Regional Transportation Authority of Pima County,"Tucson, AZ",DR,Bus,2024-05-01,8977.0


In [9]:
# read in ridership data
ridership = pd.read_excel('data/ridership.xlsx',sheet_name='VRM')
ridership = ridership[ridership['Mode/Type of Service Status'] == 'Active'].drop(['Legacy NTD ID','Reporter Type',
                                                                                  'Mode/Type of Service Status','UACE CD',
                                                                                  'TOS'],axis=1)
ridership.rename({'NTD ID':'id','Agency':'agency','3 Mode':'mode_3',
             'UZA Name':'metro_area','Mode':'mode'},axis=1,inplace=True)

In [10]:
# Pivot the table since the months are used as columns right now
ridership_melted = pd.melt(ridership,
                       id_vars=['id','agency','metro_area','mode','mode_3'], # These columns will stay the same
                       var_name='month', # Newly produced columns will be renamed 'month'
                       value_name='VRM' # The values will be in the UPT column
                       ).dropna()
# ensure month is registered as datetime for ease of future anaylsis
ridership_melted['month'] = pd.to_datetime(ridership_melted['month'])
# Need to also make sure id is still an int and not a float now with NULL values removed
ridership_melted['id'] = ridership_melted['id'].astype('int')

/tmp/ipykernel_19843/1907436874.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  ridership_melted['month'] = pd.to_datetime(ridership_melted['month'])


In [11]:
# Testing it visually with results from the R table
ridership_melted[(ridership_melted['agency']=='Regional Transportation Authority of Pima County')&(ridership_melted['mode']=='DR')].sort_values(by='month',ascending=False).head(3)

,id,agency,metro_area,mode,mode_3,month,VRM
425461,91122,Regional Transportation Authority of Pima County,"Tucson, AZ",DR,Bus,2024-07-01,98946.0
423891,91122,Regional Transportation Authority of Pima County,"Tucson, AZ",DR,Bus,2024-06-01,104208.0
422321,91122,Regional Transportation Authority of Pima County,"Tucson, AZ",DR,Bus,2024-05-01,101850.0


In [12]:
trips_melted

,id,agency,metro_area,mode,mode_3,month,UPT
0,1,King County,"Seattle--Tacoma, WA",DR,Bus,2002-01-01,135144.0
3,1,King County,"Seattle--Tacoma, WA",MB,Bus,2002-01-01,6045861.0
6,1,King County,"Seattle--Tacoma, WA",TB,Bus,2002-01-01,2044815.0
7,1,King County,"Seattle--Tacoma, WA",VP,Bus,2002-01-01,174040.0
8,2,Spokane Transit Authority,"Spokane, WA",DR,Bus,2002-01-01,23023.0
...,...,...,...,...,...,...,...
425461,91122,Regional Transportation Authority of Pima County,"Tucson, AZ",DR,Bus,2024-07-01,10811.0
425462,91122,Regional Transportation Authority of Pima County,"Tucson, AZ",MB,Bus,2024-07-01,14677.0
425463,99422,San Joaquin Council,"Stockton, CA",VP,Bus,2024-07-01,70322.0
425464,99423,City of Glendale,"Los Angeles--Long Beach--Anaheim, CA",DR,Bus,2024-07-01,2343.0


In [13]:
# Seeing which is larger for the inner-join
print(trips_melted.shape, ridership_melted.shape)

(283005, 7) (280722, 7)


In [119]:
# creating usage
usage = trips_melted.merge(ridership_melted,how='inner').dropna().reset_index(drop=True)

In [120]:
# removing mode_3 col
usage.drop('mode_3',axis=1,inplace=True)
# Only want 2022 data since our financial data is from then
usage = usage[usage['month'].dt.year == 2022]

In [126]:
usage = usage.groupby(['id','agency','metro_area','mode']).sum(['VRM']).reset_index()

In [129]:
usage[usage['agency']=='King County']

,id,agency,metro_area,mode,UPT,VRM
0,1,King County,"Seattle--Tacoma, WA",DR,663009.0,12860448.0
1,1,King County,"Seattle--Tacoma, WA",FB,400407.0,51236.0
2,1,King County,"Seattle--Tacoma, WA",MB,53983641.0,61632644.0
3,1,King County,"Seattle--Tacoma, WA",SR,1117605.0,180369.0
4,1,King County,"Seattle--Tacoma, WA",TB,9575043.0,2635705.0
5,1,King County,"Seattle--Tacoma, WA",VP,702709.0,4413910.0


In [127]:
print(usage.shape, financials.shape)

(1141, 6) (1348, 5)


In [123]:
financials

,id,agency,mode,fares,expenses
0,1,King County Department of Metro Transit,CB,5216912,0
1,1,King County Department of Metro Transit,DR,740726,60540676
2,1,King County Department of Metro Transit,DR,91601,60540676
3,1,King County Department of Metro Transit,FB,1715265,8899659
4,1,King County Department of Metro Transit,LR,29386480,0
...,...,...,...,...,...
1343,99423,City of Glendale,MB,238992,10012370
1344,99424,City of Pasadena,DR,18305,1752583
1345,99424,City of Pasadena,MB,482055,6777893
1346,99425,Pomona Valley Transportation Authority,DR,80294,3202374


In [132]:
usage.drop('agency',axis=1,inplace=True)

In [133]:
df = financials.merge(usage,how='left',on=['id','mode']).dropna()

In [134]:
# equivelant to USAGE_AND_FINANCIALS from the original code
df

,id,agency,mode,fares,expenses,metro_area,UPT,VRM
1,1,King County Department of Metro Transit,DR,740726,60540676,"Seattle--Tacoma, WA",663009.0,12860448.0
2,1,King County Department of Metro Transit,DR,91601,60540676,"Seattle--Tacoma, WA",663009.0,12860448.0
3,1,King County Department of Metro Transit,FB,1715265,8899659,"Seattle--Tacoma, WA",400407.0,51236.0
5,1,King County Department of Metro Transit,MB,56566150,671653000,"Seattle--Tacoma, WA",53983641.0,61632644.0
6,1,King County Department of Metro Transit,MB,280187,671653000,"Seattle--Tacoma, WA",53983641.0,61632644.0
...,...,...,...,...,...,...,...,...
1343,99423,City of Glendale,MB,238992,10012370,"Los Angeles--Long Beach--Anaheim, CA",624155.0,868128.0
1344,99424,City of Pasadena,DR,18305,1752583,"Los Angeles--Long Beach--Anaheim, CA",38412.0,136655.0
1345,99424,City of Pasadena,MB,482055,6777893,"Los Angeles--Long Beach--Anaheim, CA",1139100.0,701730.0
1346,99425,Pomona Valley Transportation Authority,DR,80294,3202374,"Los Angeles--Long Beach--Anaheim, CA",76187.0,725488.0


## Save all relevant files

In [135]:
financials

,id,agency,mode,fares,expenses
0,1,King County Department of Metro Transit,CB,5216912,0
1,1,King County Department of Metro Transit,DR,740726,60540676
2,1,King County Department of Metro Transit,DR,91601,60540676
3,1,King County Department of Metro Transit,FB,1715265,8899659
4,1,King County Department of Metro Transit,LR,29386480,0
...,...,...,...,...,...
1343,99423,City of Glendale,MB,238992,10012370
1344,99424,City of Pasadena,DR,18305,1752583
1345,99424,City of Pasadena,MB,482055,6777893
1346,99425,Pomona Valley Transportation Authority,DR,80294,3202374


In [136]:
fares.to_csv('data/2022_fares_cleaned.csv',index=None)
expenses.to_csv('data/2022_expenses_cleaned.csv',index=None)
financials.to_csv('data/combined_2022_financial_info.csv',index=None)
usage.to_csv('data/usage_2022.csv',index=None)
df.to_csv('data/usage_and_financials_2022.csv',index=None)